In [1]:
import torch
import sklearn
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
data = pd.read_csv("ML_data2.csv",header=None)
data
data = data.to_numpy()

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_valtest, y_train, y_valtest = train_test_split(data[:,:-1], data[:,-1],
                                                                                  train_size=0.5, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, train_size=0.5, shuffle=False)

In [31]:
y_train = np.where(y_train<=0,0,1)
y_val = np.where(y_val<=0,0,1)
y_test = np.where(y_test<=0,0,1)

In [32]:
from torch.utils.data import TensorDataset



X_train_tensor = TensorDataset(torch.Tensor(X_train),torch.Tensor(y_train))
X_val_tensor = TensorDataset(torch.Tensor(X_val),torch.Tensor(y_val))
X_test_tensor = TensorDataset(torch.Tensor(X_test),torch.Tensor(y_test))

In [37]:
# creating dataloaders for the datasets

# arguments for the dataloaders
kwargs = {'num_workers': 1, 'pin_memory': True}

batch_size = 100
# dataloaders
train_loader = DataLoader(dataset=X_train_tensor, batch_size=batch_size, shuffle=False, **kwargs)
valid_loader = DataLoader(dataset=X_val_tensor, batch_size=batch_size, shuffle=False, **kwargs)
test_loader  = DataLoader(dataset=X_test_tensor ,  batch_size=batch_size, shuffle=False,  **kwargs)

In [38]:
batch_x, batch_y = next(iter(train_loader))
print(batch_x.shape, batch_y.shape)

torch.Size([100, 4]) torch.Size([100])


In [45]:
class Network(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super().__init__()
        self.linear1 = nn.Linear(input_size, hidden_size1)
        self.linear2 = nn.Linear(hidden_size1, hidden_size2)
        self.linear3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = torch.sigmoid(self.linear3(x))
        return x


In [46]:
hidden_dim_1 = 64
hidden_dim_2 = 64

lr = 1e-3

epochs = 150

network = Network(4,hidden_dim_1,hidden_dim_2,1)

loss_function = F.binary_cross_entropy
optimizer = Adam(network.parameters(), lr=lr)


In [47]:
network.train()

print("training network...")
for epoch in range(epochs):
    train_overall_loss = 0

    valid_overall_loss = 0

    for batch_idx, (x, y) in enumerate(train_loader):
        x = x.to(device)

        optimizer.zero_grad()

        out = network(x)
        loss = loss_function(out,y.reshape(-1,1))

        train_overall_loss += loss.item()

        loss.backward()
        optimizer.step()

    # validation
    with torch.no_grad():
      for valid_batch_idx, (valid_x, valid_y) in enumerate(valid_loader):
        valid_x = valid_x.to(device)

        # optimizer.zero_grad()

        valid_out = network(valid_x)
        valid_loss = loss_function(valid_out, valid_y.reshape(-1,1))

        valid_overall_loss += valid_loss.item()

    train_n_datapoints = batch_idx * batch_size
    valid_n_datapoints = valid_batch_idx * batch_size

    print("\tEpoch", epoch + 1, " train\tAverage Loss: ", train_overall_loss / train_n_datapoints)
    print("\tEpoch", epoch + 1, " valid\tAverage Loss: ", valid_overall_loss / valid_n_datapoints)
    # # stopping criterion
    # if(len(train_elbo)>5):
    #   if(train_elbo[-6]-train_elbo[-1]<0.005):
    #     break
print("Training complete!")
# torch.save(vae,"./model/vae_adam_l40_e100.pt")

training network...
	Epoch 1  train	Average Loss:  0.13923038046700614
	Epoch 1  valid	Average Loss:  0.1170101676267736
	Epoch 2  train	Average Loss:  0.07589639057431902
	Epoch 2  valid	Average Loss:  0.029619554351357852
	Epoch 3  train	Average Loss:  0.04317743395056043
	Epoch 3  valid	Average Loss:  0.03086438312250025
	Epoch 4  train	Average Loss:  0.033954346333231245
	Epoch 4  valid	Average Loss:  0.012188010426128612
	Epoch 5  train	Average Loss:  0.018873175723212107
	Epoch 5  valid	Average Loss:  0.033687948198879464
	Epoch 6  train	Average Loss:  0.024774311661720276
	Epoch 6  valid	Average Loss:  0.007106264408896951
	Epoch 7  train	Average Loss:  0.010083911010197231
	Epoch 7  valid	Average Loss:  0.009170269159709706
	Epoch 8  train	Average Loss:  0.011524397909641266
	Epoch 8  valid	Average Loss:  0.03492107755997602
	Epoch 9  train	Average Loss:  0.03539837086200714
	Epoch 9  valid	Average Loss:  0.034444313470055075
	Epoch 10  train	Average Loss:  0.015445027726037161

In [ ]:
vae.eval()

x_original_list = []
y_list = []
x_reconstr_list = []
mu_list = []
log_sigma_list = []
with torch.no_grad():
    for batch_idx, (x, y) in enumerate(tqdm(test_loader)):
        x = x.to(device)

        x_reconstr, mu, log_sigma = vae(x)
        x_original_list.append(x)
        y_list.append(y)
        x_reconstr_list.append(x_reconstr)
        mu_list.append(mu)
        log_sigma_list.append(log_sigma)

In [ ]:
plt.figure()
plt.plot(range(len(train_elbo)),train_elbo,label="train")
plt.plot(range(len(valid_elbo)),valid_elbo,label="valid")
plt.legend()
plt.xlabel("epochs")
plt.ylabel("train elbo")
plt.show()